In [1]:
import altair as alt
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

folder = "distance_side-50_dens-0.7_pop-0.3_hom-0.3_town-True"

csv_list = os.listdir(folder)
results = pd.read_csv(folder+"/"+csv_list[0])

for csv in csv_list[1:]:
    temp = pd.read_csv(folder+"/"+csv)
    results = pd.concat([results,temp])

results["model"] = results["mobility"].apply(lambda m: eval(m)["model"])
results["beta"] = results["mobility"].apply(lambda m: eval(m)["beta"] if "beta" in m else 0)

group_gravity= results.groupby(["beta"]).agg(mean_step=('Step', np.mean), mean_seg=(
'total_segregation', np.mean)).reset_index()

seg_min, seg_max = group_gravity["mean_seg"].min(), group_gravity["mean_seg"].max()
step_min, step_max = group_gravity["mean_step"].min(), group_gravity["mean_step"].max()

clas = alt.Chart(group_gravity[group_gravity["beta"]==0]).mark_point(size=100, color='red').encode(
      x = alt.X('mean_step', scale=alt.Scale(domain=[step_min, step_max]),axis=alt.Axis(title='Max Steps')),
    y = alt.Y('mean_seg', scale=alt.Scale(domain=[seg_min, seg_max]), axis=alt.Axis(title='Segregation'))).properties(
    width=300,  height=300)

text = alt.Chart({'values':[{'x': 100, 'y': 0.8015}]}).mark_text(
    text='Classic Schelling', angle=0, size=14
).encode(x='x:Q', y='y:Q')


df = pd.DataFrame({    'x': [100, group_gravity[group_gravity["beta"]==0]["mean_step"].values[0]],
                       'y': [0.8, group_gravity[group_gravity["beta"]==0]["mean_seg"].values[0]]})
arr = alt.Chart(df).mark_line(color="red").encode(x='x',    y='y',)


In [2]:
from scipy.optimize import curve_fit

b1 = alt.Chart(group_gravity).mark_circle(size=50).encode(
      x = alt.X('mean_step', scale=alt.Scale(domain=[step_min, step_max]),axis=alt.Axis(title='Max Steps')),
    y = alt.Y('mean_seg', scale=alt.Scale(domain=[seg_min, seg_max]), axis=alt.Axis(title='Segregation')),
    color=alt.Color('beta', scale=alt.Scale(scheme='viridis'), legend=alt.Legend(title="β"))
).properties(
    width=300,
    height=300)

def func(x, a, b, c):
    return c+ a/(x**b)

xdata = group_gravity["mean_step"]
ydata =  group_gravity["mean_seg"]

popt, pcov = curve_fit(func, xdata, ydata)
pred = pd.DataFrame()
pred["xv"] = xdata
pred["yv"] = func(xdata, *popt)

print("c+ a/(x**b)")
print(popt)

regb1 = alt.Chart(pred).mark_line(color="grey", opacity=1, width=2, strokeDash=[10,5]).encode(x="xv", y="yv").properties(
    width=300,
    height=300
)

#####################################################

b2 = alt.Chart(group_gravity).mark_circle(size=50).encode(
      x = alt.X('beta', scale=alt.Scale(domain=[-5, 5]), axis=alt.Axis(title='Beta')),
    y = alt.Y('mean_step', scale=alt.Scale(domain=[step_min, step_max]), axis=alt.Axis(title='Max Steps')),
#color=alt.Color('mean_seg', scale=alt.Scale(scheme='redblue'), legend=alt.Legend(title="seg"))
).properties(
    width=300,
    height=300
)

def func(x, a, b,c,d):
    return a*(d**(-b*x))+c

xdata = group_gravity["beta"]
ydata =  group_gravity["mean_step"]

popt, pcov = curve_fit(func, xdata, ydata)
pred = pd.DataFrame()
pred["xv"] = xdata
pred["yv"] = func(xdata, *popt)

print("a*(np.e**(-b*x))+c")
print(popt)

regb2 = alt.Chart(pred).mark_line(color="grey", opacity=1, width=2, strokeDash=[10,5]).encode(x="xv", y="yv").properties(
    width=300,
    height=300
)

#####################################################

b3 = alt.Chart(group_gravity).mark_circle(size=50).encode(
      x = alt.X('beta', scale=alt.Scale(domain=[-5,5]), axis=alt.Axis(title='Beta')),
    y = alt.Y('mean_seg', scale=alt.Scale(domain=[seg_min, seg_max]), axis=alt.Axis(title='Segregation')),
#color=alt.Color('mean_step', scale=alt.Scale(scheme='redblue'), legend=alt.Legend(title="ste"))
).properties(
    width=300,
    height=300
)

def func(x, a, b, c, d, e, f,g):
    return a/((b*(g**(-c*x))+d)**e)+f

xdata = group_gravity["beta"]
ydata =  group_gravity["mean_seg"]

popt, pcov = curve_fit(func, xdata, ydata, maxfev =2500)
pred = pd.DataFrame()
pred["xv"] = xdata
pred["yv"] = func(xdata, *popt)

print("a/((b*(g**(-c*x))+d)**e)+f")
print(popt)

regb3 = alt.Chart(pred).mark_line(color="grey", opacity=1, width=2, strokeDash=[10,5]).encode(x="xv", y="yv").properties(
    width=300,
    height=300)



c+ a/(x**b)
[4.64269139 1.44558653 0.73314454]
a*(np.e**(-b*x))+c
[ 2.51752908  1.40061301 17.04560976  1.97652959]
a/((b*(g**(-c*x))+d)**e)+f
[9.70164757e-01 7.49570736e-03 2.79051094e+00 1.16040782e+00
 1.79466438e+01 7.33608840e-01 1.35851568e+00]


In [3]:



folder = "relevance_side-50_dens-0.7_pop-0.3_hom-0.3_town-True"

csv_list = os.listdir(folder)
results = pd.read_csv(folder+"/"+csv_list[0])

for csv in csv_list[1:]:
    temp = pd.read_csv(folder+"/"+csv)
    results = pd.concat([results,temp])


results["model"] = results["mobility"].apply(lambda m: eval(m)["model"])
results["metric"] = results["mobility"].apply(lambda m: eval(m)["metric"] if "metric" in m else 0)
results["alpha"] = results["mobility"].apply(lambda m: eval(m)["alpha"] if "alpha" in m else 0)


group_gravityalfa = results.groupby(["alpha"]).agg(mean_step=('Step', np.mean), 
                                                               mean_seg=('total_segregation', np.mean),
                                                               mean_cen_seg=('center_segregation', np.mean),
                                                                mean_per_seg=('periphery_segregation', np.mean),
                                                              ).reset_index()





#seg_min, seg_max = group_gravityalfa["mean_seg"].min(), group_gravityalfa["mean_seg"].max()
step_min, step_max = group_gravityalfa["mean_step"].min(), group_gravityalfa["mean_step"].max()

###########################################


seg_min_2, seg_max_2 = group_gravityalfa["mean_seg"].min(), group_gravityalfa["mean_seg"].max()

a1 = alt.Chart(group_gravityalfa).mark_circle(size=50).encode(
      x = alt.X('mean_step', scale=alt.Scale(domain=[step_min, step_max]),axis=alt.Axis(title='Max Steps')),
    y = alt.Y('mean_seg', scale=alt.Scale(domain=[seg_min_2, seg_max_2]), axis=alt.Axis(title='Segregation')),
    color=alt.Color('alpha', scale=alt.Scale(scheme='viridis'), legend=alt.Legend(title="α"))
).properties(
    width=300,
    height=300)


def func(x, a, b, c):
    return c+ a/(x**b)


xdata = group_gravityalfa["mean_step"]
ydata =  group_gravityalfa["mean_seg"]

popt, pcov = curve_fit(func, xdata, ydata)
pred = pd.DataFrame()
pred["xv"] = xdata
pred["yv"] = func(xdata, *popt)

print("c+ a/(x**b)")
print(popt)

rega1 = alt.Chart(pred).mark_line(color="grey", opacity=1, width=2, strokeDash=[10,5]).encode(x="xv", y="yv").properties(
    width=300,
    height=300
)

###########################################


a2 = alt.Chart(group_gravityalfa).mark_circle(size=50).encode(
      x = alt.X('alpha', scale=alt.Scale(domain=[0, 3]),axis=alt.Axis(title='Alpha')),
    y = alt.Y('mean_step', scale=alt.Scale(domain=[step_min, step_max]), axis=alt.Axis(title='Max Steps'))
).properties(
    width=300,
    height=300)


def func(x, a, b,c,d):
    return a*(d**(b*x))+c


xdata = group_gravityalfa["alpha"]
ydata =  group_gravityalfa["mean_step"]

popt, pcov = curve_fit(func, xdata, ydata)
pred = pd.DataFrame()
pred["xv"] = xdata
pred["yv"] = func(xdata, *popt)

print("a*(d**(b*x))+c")
print(popt)

rega2 = alt.Chart(pred).mark_line(color="grey", opacity=1, width=2, strokeDash=[10,5]).encode(x="xv", y="yv").properties(
    width=300,
    height=300
)


###########################################


a3 = alt.Chart(group_gravityalfa).mark_circle(size=50).encode(
      x = alt.X('alpha', scale=alt.Scale(domain=[0, 3]),axis=alt.Axis(title='Alpha')),
    y = alt.Y('mean_seg', scale=alt.Scale(domain=[seg_min, seg_max]), axis=alt.Axis(title='Segregation'))
).properties(
    width=300,
    height=300)


def func(x, a, b,c):
    return a*x**2+b*x+c


xdata = group_gravityalfa["alpha"]
ydata =  group_gravityalfa["mean_seg"]

popt, pcov = curve_fit(func, xdata, ydata)
pred = pd.DataFrame()
pred["xv"] = xdata
pred["yv"] = func(xdata, *popt)

print("a*x**2+b*x+c")
print(popt)

rega3 = alt.Chart(pred).mark_line(color="grey", opacity=1, width=2, strokeDash=[10,5]).encode(x="xv", y="yv").properties(
    width=300,
    height=300
)

c+ a/(x**b)
[-8.50573148  2.08784986  0.81149487]
a*(d**(b*x))+c
[ 1.65401032  1.68536905 15.65641703  2.7011682 ]
a*x**2+b*x+c
[-0.00168834  0.01223414  0.79114289]


In [4]:
data = alt.Data(values=[{'x': 'A'}])
a = (alt.Chart(data).mark_text(text='a)', size=15, x=0, y=-20, dx = 0, dy= 0))
b = (alt.Chart(data).mark_text(text='b)', size=15, x=0, y=-20, dx = 0, dy=0))
c = (alt.Chart(data).mark_text(text='c)',  size=15, x=0, y=-20, dx = 0, dy= 0))
d = (alt.Chart(data).mark_text(text='d)',  size=15, x=0, y=-20, dx = 0, dy= 0))
e = (alt.Chart(data).mark_text(text='e)',  size=15, x=0, y=-20, dx = 0, dy= 0))
f = (alt.Chart(data).mark_text(text='f)',  size=15, x=0, y=-20, dx = 0, dy= 0))

latext = alt.Chart({'values':[{'x': 250, 'y': 0.78}]}).mark_text(
    text="y=c+a/xᵇ", angle=0, size=20
).encode(
    x='x:Q', y='y:Q'
)

latext2 = alt.Chart({'values':[{'x': 250, 'y': 0.77}]}).mark_text(
    text="a=4.6, b=1.4, c=0.7", angle=0, size=14
).encode(
    x='x:Q', y='y:Q'
)

clas2 = alt.Chart(group_gravityalfa[group_gravityalfa["alpha"]==0]).mark_point(size=100, color='red').encode(
      x = alt.X('mean_step', scale=alt.Scale(domain=[step_min, step_max]),axis=alt.Axis(title='Max Steps')),
    y = alt.Y('mean_seg', scale=alt.Scale(domain=[seg_min_2, seg_max_2]), axis=alt.Axis(title='Segregation'))).properties(
    width=300,  height=300)

text2 = alt.Chart({'values':[{'x': 100, 'y': 0.8015}]}).mark_text(
    text='Classic Schelling', angle=0, size=14
).encode(x='x:Q', y='y:Q')



df = pd.DataFrame({    'x': [100, group_gravityalfa[group_gravityalfa["alpha"]==0]["mean_step"].values[0]],
                       'y': [0.8, group_gravityalfa[group_gravityalfa["alpha"]==0]["mean_seg"].values[0]]})
arr2 = alt.Chart(df).mark_line(color="red").encode(x='x',    y='y',)


In [5]:
seg_min_2, seg_max_2

(0.7922044553761124, 0.8124306284404788)

In [6]:
a1

alt.Chart(...)

In [7]:
alt.vconcat(
    
    alt.hconcat((b1 + regb1 + clas + text + arr+a)   ,    (b2+regb2+b),  (b3+regb3+c) ).resolve_scale(
    color='independent') &
    
    alt.hconcat((a1 + rega1 + clas2 + text2 + arr2 +d),   (a2+rega2+e), (a3+rega3+f)).resolve_scale(
    color='independent')
    
).resolve_scale(
    color='independent') 

alt.VConcatChart(...)

In [58]:
import altair as alt


folder = "gravity_side-50_dens-0.7_pop-0.3_hom-0.3_town-True"

csv_list = os.listdir(folder)
results = pd.read_csv(folder+"/"+csv_list[0])

for csv in csv_list[1:]:
    temp = pd.read_csv(folder+"/"+csv)
    results = pd.concat([results,temp])


results["model"] = results["mobility"].apply(lambda m: eval(m)["model"])
results["metric"] = results["mobility"].apply(lambda m: eval(m)["metric"] if "metric" in m else 0)
results["beta"] = results["mobility"].apply(lambda m: eval(m)["beta"] if "beta" in m else 0)
results["alpha"] = results["mobility"].apply(lambda m: eval(m)["alpha"] if "alpha" in m else 0)


group_gravityalfabeta = results.groupby(["alpha","beta"]).agg(mean_step=('Step', np.mean), 
                                                               mean_seg=('total_segregation', np.mean),
                                                               mean_cen_seg=('center_segregation', np.mean),
                                                                mean_per_seg=('periphery_segregation', np.mean),
                                                              ).reset_index()


#group_gravityalfabeta = group_gravityalfabeta[group_gravityalfabeta["alpha"]<=2group_gravityalfabeta = group_gravityalfabeta[group_gravityalfabeta["alpha"]<=2#2group_gravityalfabeta = group_gravityalfabeta[group_gravityalfabeta["alpha"]<=2group_gravityalfabeta = group_gravityalfabeta[group_gravityalfabeta["alpha"]<=2#]
group_gravityalfabeta = group_gravityalfabeta[group_gravityalfabeta["alpha"] == group_gravityalfabeta["alpha"].apply(lambda x: int(x))]



heatmap1 = alt.Chart(group_gravityalfabeta).mark_rect().encode(
    alt.X('beta:O'),
    alt.Y('alpha:O', scale=alt.Scale(reverse=True)),
    
    color=alt.Color('mean_seg:Q',
        scale=alt.Scale(scheme='reds'),
        legend=alt.Legend(title="Segregation"),
    )).properties(
    width=300,
    height=300)


heatmap2 = alt.Chart(group_gravityalfabeta).mark_rect().encode(
    alt.X('beta:O'),
    alt.Y('alpha:O', scale=alt.Scale(reverse=True)),
    
    color=alt.Color('mean_step:Q',
        scale=alt.Scale(scheme='blues'),
        legend=alt.Legend(title="Max Steps"),
    )).properties(
    width=300,
    height=300)



In [59]:
(heatmap1 | heatmap2).resolve_scale(
    color='independent') 

alt.HConcatChart(...)

In [60]:
alpha_mean = group_gravityalfabeta.groupby("alpha").mean().reset_index()


beta_mean = group_gravityalfabeta.groupby("beta").mean().reset_index()



In [61]:
bar1 =alt.Chart(alpha_mean).mark_rect().encode(alt.X('alpha:O'),
    alt.Y('beta', scale=alt.Scale(reverse=True), axis=None),
    color=alt.Color('mean_seg:Q',    scale=alt.Scale(scheme='reds'),   
    )).properties(width=250,height=50)

bar2 =alt.Chart(beta_mean).mark_rect().encode(alt.X('beta:O'),
    alt.Y('alpha', scale=alt.Scale(reverse=True), axis=None),
    color=alt.Color('mean_seg:Q',    scale=alt.Scale(scheme='reds'),    legend=None,
    )).properties(width=250,height=50)

bar3 =alt.Chart(alpha_mean).mark_rect().encode(alt.X('alpha:O'),
    alt.Y('beta', scale=alt.Scale(reverse=True), axis=None),
    color=alt.Color('mean_step:Q',    scale=alt.Scale(scheme='blues'),   
    )).properties(width=250,height=50)

bar4 =alt.Chart(beta_mean).mark_rect().encode(alt.X('beta:O'),
    alt.Y('alpha', scale=alt.Scale(reverse=True), axis=None),
    color=alt.Color('mean_step:Q',    scale=alt.Scale(scheme='blues'),    legend=None,
    )).properties(width=250,height=50)

alt.vconcat(bar1, bar2, bar3, bar4).resolve_scale(
    color='independent') 

alt.VConcatChart(...)

In [30]:
heatmap1 

alt.Chart(...)

In [18]:
heatmap2

alt.Chart(...)

In [47]:
alt.vconcat(
    
    alt.hconcat((b1 + regb1 + clas + text + arr+a)   ,    (b2+regb2+b),  (b3+regb3+c) ).resolve_scale(
    color='independent') &
    
    alt.hconcat((a1 + rega1 + clas2 + text2 + arr2 +d),   (a2+rega2+e), (a3+rega3+f)).resolve_scale(
    color='independent') &
    
    alt.hconcat(heatmap1, heatmap2,
           alt.vconcat(bar1, bar2, bar3, bar4).resolve_scale(
    color='independent') ).resolve_scale(
    color='independent') 
    
).resolve_scale(
    color='independent') 

alt.VConcatChart(...)

In [69]:
b2
rule = alt.Chart(group_gravity).mark_rule(color="black", opacity=1, width=2, strokeDash=[10,5]).encode(
    x="mean(beta)"
)

In [70]:
b2+rule

alt.LayerChart(...)

In [27]:
heatmap2 = alt.Chart(group_gravityalfabeta).mark_rect().encode(
    alt.X('beta:O'),
    alt.Y('alpha:O', scale=alt.Scale(reverse=True)),
    
    color=alt.Color('mean_step:Q',
        scale=alt.Scale(scheme='cividis'),
        legend=alt.Legend(title="Max Steps"),
    )).properties(
    width=250,
    height=250)





heatmap2

alt.Chart(...)

In [34]:
group_gravityalfabeta["diffseg"] = group_gravityalfabeta["periphery"] - group_gravityalfabeta["center"]

5     0.007605
11    0.045476
17    0.062956
23    0.063843
29    0.069702
35    0.073684
Name: diffseg, dtype: float64

In [39]:
import altair as alt
import pandas as pd

source = pd.DataFrame({
    'a': group_gravityalfabeta[group_gravityalfabeta["beta"]==0]["alpha"],
    'b':group_gravityalfabeta[group_gravityalfabeta["beta"]==0]["diffseg"]
})

alt.Chart(source).mark_bar().encode(
    x='a',
    y='b'
)

alt.Chart(...)

In [ ]:
alt.Chart(df0.melt('beta')).mark_bar().encode(
    alt.X('variable:N', axis=None),
    alt.Y('value:Q', axis=alt.Axis(title='alpha = 0'),  scale=alt.Scale(domain=[0.6, 0.85])),
    color=alt.Color('variable:N', 
                     #legend = alt.Legend(values=["a","b"]),
                    scale=alt.Scale(domain=["center","periphery"], range=["firebrick", "#fcc40f"])),
column=alt.Column('beta', spacing=10, title=None, 
                  header=alt.Header(labels=False))
)

In [28]:
group_gravityalfabeta.rename(columns={"mean_cen_seg": "center",
                                     "mean_per_seg": "periphery"}, inplace=True)

df0 = group_gravityalfabeta[group_gravityalfabeta["alpha"]==0][["beta","center","periphery"]]

p0 = alt.Chart(df0.melt('beta')).mark_bar().encode(
    alt.X('variable:N', axis=None),
    alt.Y('value:Q', axis=alt.Axis(title='alpha = 0'),  scale=alt.Scale(domain=[0.6, 0.85])),
    color=alt.Color('variable:N', 
                     #legend = alt.Legend(values=["a","b"]),
                    scale=alt.Scale(domain=["center","periphery"], range=["firebrick", "#fcc40f"])),
column=alt.Column('beta', spacing=10, title=None, 
                  header=alt.Header(labels=False))
).properties(
    width=100,
    height=25

)





df2 = group_gravityalfabeta[group_gravityalfabeta["alpha"]==1][["beta","center","periphery"]]

p2 = alt.Chart(df2.melt('beta')).mark_bar().encode(
    alt.X('variable:N', axis=None),
    alt.Y('value:Q', axis=alt.Axis(title='alpha = 1'),  scale=alt.Scale(domain=[0.6, 0.85])),
    color=alt.Color('variable:N',  scale=alt.Scale(scheme='oranges')) ,
    column=alt.Column('beta', spacing=10, title=None, header=alt.Header(labels=False))

).properties(
    width=100,
    height=25
)


df4 = group_gravityalfabeta[group_gravityalfabeta["alpha"]==2][["beta","center","periphery"]]

p4 = alt.Chart(df4.melt('beta')).mark_bar().encode(
    alt.X('variable:N', axis=None),
    alt.Y('value:Q', axis=alt.Axis(title='alpha = 2'),  scale=alt.Scale(domain=[0.6, 0.85])),
    color=alt.Color('variable:N',  scale=alt.Scale(scheme='oranges')) ,
    #column='beta',
column=alt.Column('beta', spacing=10, title=None, header=alt.Header(labelPadding = -120))
).properties(
    width=100,
    height=25)

In [29]:
data = alt.Data(values=[{'x': 'A'}])
a = (alt.Chart(data).mark_text(text='a)', size=15, x=0, y=-20, dx = 0, dy= 0))
b = (alt.Chart(data).mark_text(text='b)', size=15, x=0, y=-20, dx = 0, dy=0))
c = (alt.Chart(data).mark_text(text='c)', size=15, x=0, y=-20, dx = 0, dy=325))

In [30]:
bottom = (p0 &p2 & p4)

In [31]:
chart= alt.vconcat(
    
    alt.hconcat(heatmap1 +a +c, heatmap2 + b ).resolve_scale(
    color='independent') &
    
   alt.hconcat(bottom ).resolve_scale(
    color='independent') 
    
).resolve_scale(
    color='independent')

chart

alt.VConcatChart(...)

In [32]:
group_gravityalfabeta.groupby("alpha").std()

,beta,mean_step,mean_seg,center,periphery
alpha,,,,,
0.0,1.870829,109.284368,0.025262,0.023626,0.025592
0.5,1.870829,104.690542,0.027697,0.015575,0.030054
1.0,1.870829,101.017655,0.029013,0.016056,0.032037
1.5,1.870829,115.832083,0.031559,0.024405,0.033633
2.0,1.870829,155.895320,0.030654,0.027310,0.032097
2.5,1.870829,137.506670,0.030598,0.033379,0.030752


In [44]:
group_gravityalfabeta.groupby("beta").std()

,alpha,mean_step,mean_seg,center,periphery
beta,,,,,
-5.0,0.935414,72.082385,0.002210,0.018707,0.005657
-4.0,0.935414,153.736298,0.004462,0.022194,0.009472
-3.0,0.935414,190.170920,0.009033,0.014103,0.013583
-2.0,0.935414,165.932210,0.009943,0.015013,0.014212
-1.0,0.935414,103.278992,0.007099,0.012555,0.010669
0.0,0.935414,47.370298,0.007937,0.014237,0.011857


In [16]:
heatmap1

alt.Chart(...)